## SET UP

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!git clone https://github.com/LauraMJanssen/srcnn_fsrcnn_srgan.git
%cd srcnn_fsrcnn_srgan

Cloning into 'srcnn_fsrcnn_srgan'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 93 (delta 32), reused 71 (delta 13), pack-reused 0
Unpacking objects: 100% (93/93), done.
/content/srcnn_fsrcnn_srgan


In [14]:
!rm -r /content/srcnn_fsrcnn_srgan
%cd /content/

/content


## TRAIN

i. SRCNN

For transfer learning: \\
remove '#' in line 31 in 'srcnn_fsrcnn_srgan/SRCNN/solver.py'

In [18]:
!python3 main.py -m srcnn \
                 --lr 0.0003 \
                 --nEpochs 20

===> Loading datasets

===> Epoch 1 starts:
 5398/5398 [================================================================================>]  Step: 12ms | Tot: 1m15s | Loss: 0.0010
    Average Loss: 0.0010
 1349/1349 [================================================================================>]  Step: 207ms | Tot: 5m52s | PSNR: 32.4992
    Average PSNR: 32.4992 dB
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
Checkpoint saved to model_path.pth


In [ ]:
!zip -r /contetn/srcnn_model.zip '/content/srcnn_fsrcnn_srgan/model_path.pth'

In [ ]:
from google.colab import files
files.download("/content/srcnn_model.zip")

ii. FSRCNN

For transfer learning: \\
remove '#' in line 29 in 'srcnn_fsrcnn_srgan/FSRCNN/solver.py'

In [ ]:
!python3 main.py -m srcnn \
                 --lr 0.0003 \
                 --nEpochs 20

In [ ]:
!zip -r /contetn/fsrcnn_model.zip '/content/srcnn_fsrcnn_srgan/model_path.pth'

In [ ]:
from google.colab import files
files.download("/content/fsrcnn_model.zip")

iii. SRGAN

In [ ]:
!python3 main.py -m srgan \
                 --lr 0.0003 \
                 --nEpochs 20

In [ ]:
!zip -r /content/mGen.zip '/content/srcnn_fsrcnn_srgan/SRGAN_Generator_model_path.pth'

  adding: content/super-resolution/SRGAN_Generator_model_path.pth (deflated 8%)


In [ ]:
!zip -r /content/mDis.zip '/content/srcnn_fsrcnn_srgan/SRGAN_Discriminator_model_path.pth'

  adding: content/super-resolution/SRGAN_Discriminator_model_path.pth (deflated 8%)


In [ ]:
from google.colab import files
files.download("/content/mGen.zip")
files.download("/content/mDis.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## TEST

(1) Define quality metric functions

In [19]:
from skimage.metrics import structural_similarity as ssim

# define a function for peak signal-to-noise ratio (PSNR)
def psnr(target, ref):
         
    # assume RGB image
    target_data = target.astype(float)
    ref_data = ref.astype(float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(np.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

# define function for mean squared error (MSE)
def mse(target, ref):
    # the MSE between the two images is the sum of the squared difference between the two images
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    
    return err

# define function that combines all three image quality metrics
def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel =True))
    
    return scores

(2) Execute model on specific image:

i. SRCNN & FSRCNN

In [20]:
!python3 super_resolve.py --input '/content/drive/MyDrive/MA/IVUS_LowRes_Scale4_Valid/005.png' \
                          --output 'test.png' \
                          --model '/content/srcnn_fsrcnn_srgan/model_path.pth'

Namespace(input='/content/drive/MyDrive/MA/IVUS_LowRes_Scale4_Valid/005.png', model='/content/srcnn_fsrcnn_srgan/model_path.pth', output='test.png')
output image saved to  test.png


ii. SRGAN

In [ ]:
!python3 super_resolve.py --input '/content/drive/MyDrive/MA/IVUS_LowRes_Scale4_Valid/005.png' \
                          --output 'test.png' \
                          --model '/content/super-resolution/SRGAN_Generator_model_path.pth'

(3) Print results \\
(make sure to adapt ref and degraded parts according to image read above)

In [22]:
import math
import numpy as np
import cv2

ref = cv2.imread('/content/drive/MyDrive/MA/IVUSImages_PNG_Valid2/001.png')
degraded = cv2.imread('/content/drive/MyDrive/MA/IVUS_LowRes_Scale4_Valid/001.png')
output = cv2.imread('/content/srcnn_fsrcnn_srgan/test.png')
  
# image quality calculations
scores = []
h, w, _ = degraded.shape
degraded2 = cv2.resize(degraded, (4*w, 4*h))
scores.append(compare_images(degraded2, ref))

scores.append(compare_images(output, ref))

  
# print all scores for all images
print('Degraded Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[0][0], scores[0][1], scores[0][2]))
print('Reconstructed Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[1][0], scores[1][1], scores[1][2]))

Degraded Image: 
PSNR: 29.40398264707939
MSE: 223.77076721191406
SSIM: 0.8560081594432712

Reconstructed Image: 
PSNR: 21.210756573209252
MSE: 1476.134593963623
SSIM: 0.5657327444253906

